In [ ]:
import time
from langserve import RemoteRunnable

# Get the remote runnable from the FastAPI server created in jockey.py
jockey = RemoteRunnable("http://localhost:8000/jockey")
# Guarantee unique session ID every time for testing
session_id = time.time()

In [ ]:
def parse_langserve_events(event: dict):
    """Used to parse events emitted from Jockey when called as an API."""
    if event["event"] == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(f"{content}", end="", flush=True)
            # Attempts to maintain a consistent token rendering rate for a smoother UX
            time.sleep(0.01)
    elif event["event"] == "on_tool_start":
        tool = event["name"]
        print(f"Running => {tool}", flush=True)
    elif event["event"] == "on_tool_end":
        tool = event["name"]
        print(f"Finished running {tool}", flush=True)

In [ ]:
tool_names = ["video-search", "download-video", "combine-clips", "remove-segment"]
### 
# Used to limit the events by type streamed from the FastAPI server when Jockey is running
# This is reduces the amount of data transmitted to the client from the server and reduces the parsing logic required.
# Most events aren't relevant for rendering the client side UI but can be added as need.
###
include_types = ["chat_model"]
include_names=["AzureChatOpenAI"] + tool_names

#### 
# We make an async call to Jockey with the `astream_events` endpoint so that we can render events and tokens as they are emitted.
# This is equivalent to making a `POST http://localhost:8000/jockey/astream_events` call
###
async for event in jockey.astream_events({"input": "Use index id 659f2e829aba4f0b402f6488 to find the top 2 clips of a touchdown."}, 
                                         {'configurable': { 'session_id': session_id }}, version="v1",
                                         include_types=include_types, include_names=include_names):
    parse_langserve_events(event)

In [ ]:
async for event in jockey.astream_events({"input": "Use index id 659f2e829aba4f0b402f6488 to find the top 2 clips of a touchdown."}, 
                                         {'configurable': { 'session_id': session_id }}, version="v1"):
    print(event)